In [ ]:
!unzip -q rice-dataset-sample.zip

In [ ]:
!pip install split-folders

In [ ]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"Dispositivo disponible: {device}")

Dispositivo disponible: cpu


## División de los conjuntos de train, val y test

In [ ]:
import splitfolders

input_dir = '/content/rice-dataset-sample'
output_dir = '/content/rice-dataset-sample-splits'

splitfolders.ratio(
    input_dir,
    output=output_dir,
    seed=42,
    ratio=(.7, .2, .1)
)

Copying files: 50 files [00:00, 4574.34 files/s]


## Instanciar DataLoaders

In [ ]:
from torchvision import transforms

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [ ]:
from torchvision import datasets

dataset_path = '/content/rice-dataset-sample-splits'

train_dataset = datasets.ImageFolder(root=f"{dataset_path}/train", transform=transform)
val_dataset = datasets.ImageFolder(root=f"{dataset_path}/val", transform=transform)
test_dataset = datasets.ImageFolder(root=f"{dataset_path}/test", transform=transform)

In [ ]:
from torch.utils.data import DataLoader

batch_size = 16
shuffle = True

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=shuffle)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=shuffle)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=shuffle)

## Instanciar modelo para transfer learning

In [ ]:
from torchvision import models

# TODO: Instancia el modelo AlexNet preentrenado.
# TIP: Consulta la documentación de "alexnet" de PyTorch
alexnet = models.alexnet(pretrained=True)

# TODO: Modifica la última capa de la red a una capa lineal
# Ten en cuenta que el número de features de entrada es el mismo que el número de entradas que tuviera en el modelo original
# Ten en cuenta que el número de features de salida debe ser 5 (porque tenemos 5 clases)
alexnet.classifier[6] = torch.nn.Linear(alexnet.classifier[6].in_features, 5)
alexnet.to(device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

## Entrenamiento de la red neuronal (transfer learning)

In [ ]:
import torch.optim as optim

criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(alexnet.parameters())

In [ ]:
num_epochs = 15

best_val_loss = float("inf")
patience = 5
patience_counter = 0

for epoch in range(num_epochs):
  alexnet.train()
  running_loss = 0.0
  for inputs, labels in train_dataloader:
    inputs, labels = inputs.to(device), labels.to(device)

    optimizer.zero_grad()

    # TODO: Calcula la salida del modelo
    outputs = alexnet(inputs)

    loss = criterion(outputs, labels)
    loss.backward
    optimizer.step()

    running_loss += loss.item()

  epoch_loss = running_loss / len(train_dataloader)

  val_loss = 0.0
  alexnet.eval()
  with torch.no_grad():
    for inputs, labels in val_dataloader:
      inputs, labels = inputs.to(device), labels.to(device)
      outputs = alexnet(inputs)
      loss = criterion(outputs, labels)
      val_loss += loss.item()

  val_loss /= len(val_dataloader)

  print(f"Epoch: {epoch+1} Train loss: {epoch_loss} Val loss: {val_loss}")

  # Early Stopping
  # TODO: Intenta programar un early_stopping.
  # TIP:  Early stopping busca parar el entrenamiento cuando la loss de validación no mejora en "n" steps.
  if val_loss < best_val_loss:
    best_val_loss = val_loss
    torch.save(alexnet.state_dict(), "best_model.pth")
    patience_counter = 0
    print("Improved model")
  else:
    patience_counter += 1

  if patience_counter >= patience:
    print("Training stopped")
    break

Epoch: 1 Train loss: 1.5057411988576253 Val loss: 1.730149507522583
Improved model
Epoch: 2 Train loss: 1.5759328206380208 Val loss: 1.730149507522583
Epoch: 3 Train loss: 1.6783963441848755 Val loss: 1.730149269104004
Improved model
Epoch: 4 Train loss: 1.6711142857869465 Val loss: 1.730149507522583
Epoch: 5 Train loss: 1.666867693265279 Val loss: 1.730149269104004
Epoch: 6 Train loss: 1.7912013530731201 Val loss: 1.730149507522583
Epoch: 7 Train loss: 1.604670763015747 Val loss: 1.730149507522583
Epoch: 8 Train loss: 1.7421029806137085 Val loss: 1.730149507522583
Training stopped


## Evaluar el modelo

In [ ]:
from sklearn.metrics import classification_report

alexnet.eval()
all_preds = []
all_labels = []

with torch.no_grad():
  for inputs, labels in train_dataloader:
    inputs, labels = inputs.to(device), labels.to(device)

    outputs = alexnet(inputs)

    _, preds = torch.max(outputs, 1)

    all_preds.extend(preds.numpy(force=True))
    all_labels.extend(labels.numpy(force=True))

print(classification_report(all_labels, all_preds, target_names=test_dataset.classes))

              precision    recall  f1-score   support

     Arborio       0.27      0.86      0.41         7
     Basmati       0.00      0.00      0.00         7
      Ipsala       0.00      0.00      0.00         7
     Jasmine       0.00      0.00      0.00         7
   Karacadag       0.00      0.00      0.00         7

    accuracy                           0.17        35
   macro avg       0.05      0.17      0.08        35
weighted avg       0.05      0.17      0.08        35



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
